In [ ]:
import os
import geopandas as gpd
import pandas as pd
import pyproj
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import folium
from folium.plugins import MousePosition

#from sqlalchemy import create_engine
#import pyproj
#import pandas as pd
#import numpy as np

In [ ]:
password = ''
db_connection_url = "postgresql://postgres:{}@localhost:5432/HSDC".format(password)
con = create_engine(db_connection_url)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
os.chdir(r'D:\iMMAP\proj\ASDC\data\flood_layer')

## Preping vector data

In [ ]:
# Define projection
repro_crs = '+proj=cea'

# Load data and convert to equal area projection
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs(repro_crs)
#flood_l1 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_029cm_dissolve_all_v2', con, 'wkb_geometry').to_crs(repro_crs)
#flood_l2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_121cm_dissolve_all_v2', con, 'wkb_geometry').to_crs(repro_crs)
flood_l3 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_271cm_dissolve_all_v2', con, 'wkb_geometry').to_crs(repro_crs)

#water_shed = gpd.GeoDataFrame.from_postgis('SELECT * from afg_sheda_lvl3', con, 'wkb_geometry').to_crs('ESRI:53034')
#build = gpd.GeoDataFrame.from_postgis('SELECT * from afg_buildings_microsoft', con).to_crs('ESRI:53034')
#pop = rio.open(r'D:\iMMAP\data\Afghanistan\afg_worldpop_2020_UNadj_unconstrained.tif')

In [ ]:
#dissolve by district and province - done
#cut by new district and province
#zonal stats on pop, build
#calculate area

In [ ]:
flood_l3_dis = flood_l3.clip(admin)
flood_l3_dis

In [ ]:
flood_l3_dis.plot()

In [ ]:
#admin2 = admin.to_crs(epsg=4326)

m = folium.Map(location=[32.2035,  69.7423], zoom_start=10)

for _, r in flood_l3.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['wkb_geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    #folium.Popup(r['district_n']).add_to(geo_j)
    geo_j.add_to(m)

MousePosition().add_to(m)


In [ ]:
m

In [ ]:
##m2 = folium.Map(location=[32.2035,  69.7423], zoom_start=10)
##
##for _, r in flood_l1_dis.to_crs(epsg=4326).iterrows():
##    # Without simplifying the representation of each borough,
##    # the map might not be displayed
##    sim_geo = gpd.GeoSeries(r['wkb_geometry']) #.simplify(tolerance=0.001)
##    geo_j = sim_geo.to_json()
##    geo_j = folium.GeoJson(data=geo_j,
##                           style_function=lambda x: {'fillColor': 'orange'})
##    #folium.Popup(r['district_n']).add_to(geo_j)
##    geo_j.add_to(m2)
##
##MousePosition().add_to(m2)
##
##m2

In [ ]:
# Save to PostGIS
#flood_l1_dis_clean.to_postgis("afg_fldzonea_100k_ncia_v2_029cm_dissolve_all", con)#